### 导包

In [1]:
import requests # 发起网络请求
import json  # 处理json格式的数据
import time  # 时间
import random  # 随机
import re  # 正则表达式
import os  # 文件操作
import pandas as pd  # 数据处理工具
import numpy as np  # 数学运算工具
import jieba  # jieba分词工具
import emoji  # 处理表情符号

### 合并所有爬取数据，并保存为csv文件

In [2]:
# 淘宝天猫
def tm():
    filenames=os.listdir('./data/tm/')
    data=[]
    for filename in filenames:
        with open(f'./data/tm/{filename}','r',encoding='utf-8') as f:
            temps=json.load(f)['rateDetail']['rateList']
            for temp in temps:
                if temp not in data:
                    data.append(temp)
    for eachone in data:
        froms.append('淘宝天猫')
        ids.append(eachone['id'])
        num=int(str(eachone['tradeEndTime']['time'])[:-3])+28800
        gou_times.append(time.strftime('%Y-%m-%d %H:%M:%S',time.gmtime(num)))
        com_times.append(eachone['rateDate'])
        usernames.append(eachone['displayUserNick'])
        skus.append(eachone['auctionSku'])
        comments.append(eachone['rateContent'])
        _=eachone['appendComment']
        if _ is not None:
            add_comments.append(_['content'])
            add_times.append(_['commentTime'])
            days.append(_['days'])
        else:
            add_comments.append('')
            add_times.append('')
            days.append('')
            
# 京东商城
def jd():
    filenames=os.listdir('./data/jd/')
    data=[]
    for filename in filenames:
        with open(f'./data/jd/{filename}','r',encoding='utf-8') as f:
            temps=json.load(f)['comments']
            for temp in temps:
                if temp not in data:
                    data.append(temp)
    for eachone in data:
        froms.append('京东商城')
        ids.append(eachone['id'])
        gou_times.append(eachone['referenceTime'])
        com_times.append(eachone['creationTime'])
        usernames.append(eachone['nickname'])
        skus.append(eachone['productColor']+' '+eachone['productSize'])
        comments.append(eachone['content'])
        if 'afterUserComment' in eachone.keys():
            add_comments.append(eachone['afterUserComment']['content'])
            add_times.append(eachone['afterUserComment']['created'])
        else:
            add_comments.append('')
            add_times.append('')
        days.append(eachone['days'])
        
# 苏宁易购
def sn():
    filenames=os.listdir('./data/sn/')
    data=[]
    for filename in filenames:
        with open(f'./data/sn/{filename}','r',encoding='utf-8') as f:
            temps=json.load(f)['commodityReviews']
            for temp in temps:
                if temp not in data:
                    data.append(temp)
    for each in data:
        froms.append('苏宁易购')
        ids.append(each['commodityReviewId'])
        gou_times.append('')
        com_times.append(each['publishTime'])
        usernames.append(each['userInfo']['nickName'])
        skus.append(each['commodityInfo']['charaterDesc1']+' '+each['commodityInfo']['charaterDesc2'])
        comments.append(each['content'])
        if 'againContent' in each['againReview'].keys():
            add_comments.append(each['againReview']['againContent'])
            add_times.append(each['againReview']['publishTime'])
            days.append(each['againReview']['publishTimeStr'])
        else:
            add_comments.append('')
            add_times.append('')
            days.append('')
            
# 华为商城
def hw():
    froms=[]
    ids=[]
    gou_times=[]
    com_times=[]
    usernames=[]
    skus=[]
    comments=[]
    add_comments=[]
    add_times=[]
    days=[]
    filenames=os.listdir('./data/hw/')
    data=[]
    for filename in filenames:
        with open(f'./data/hw/{filename}','r',encoding='utf-8') as f:
            temps=json.load(f)['data']['comments']
            for temp in temps:
                if temp not in data:
                    data.append(temp)
    codes=[]
    for each in data:
        froms.append('华为商城')
        ids.append(each['commentId'])
        gou_times.append('')
        com_times.append(each['creationTime'])
        usernames.append(each['userName'])
        codes.append(each['skuCode'])
        try:
            skus.append(each['skuAttrs'])
        except:
            skus.append('')
        comments.append(each['content'])
        add_comments.append('')
        add_times.append('')
        days.append('')
    columns=['平台','评价id','购买日期','评论日期','用户名','购买手机的配置信息','购买代码','评价内容','追评内容','追评日期','追评相隔的时间']
    df=pd.DataFrame([froms,ids,gou_times,com_times,usernames,skus,codes,comments,add_comments,add_times,days]).T
    df.columns=columns
    df1=df.copy()
    codes=list(df['购买代码'].value_counts().index)
    for code in codes:
        temp=df[df['购买代码']==code]['购买手机的配置信息'].value_counts()
        sku=temp.index[0]
        if len(temp)>1 and sku == '':
            sku=temp.index[1]
        elif sku == '':
            df1.drop(list(df[df['购买代码']==code].index),inplace=True)
            continue
        df1.loc[list(df[df['购买代码']==code].index),'购买手机的配置信息']=sku
    df1.drop(columns='购买代码',inplace=True)
    index=np.arange(0,len(df1))
    df1.index=index
    return df1

froms=[]  # 电商平台
ids=[]  # 评价id
gou_times=[] # 购买日期
com_times=[] # 评论日期
usernames=[] # 用户名
skus=[]  # 购买手机的配置信息
comments=[] # 评价内容
add_comments=[] # 追评内容
add_times=[] # 追评日期
days=[]  # 追评相隔的时间

tm()
jd()
sn()
columns=['平台','评价id','购买日期','评论日期','用户名','购买手机的配置信息','评价内容','追评内容','追评日期','追评相隔的时间']
df=pd.DataFrame([froms,ids,gou_times,com_times,usernames,skus,comments,add_comments,add_times,days]).T
df.columns=columns
df1=hw()
df=pd.concat([df,df1],ignore_index=True)
# 保存数据
df.to_csv('./data/全部数据.csv',index=False,encoding='utf_8_sig')

### 整体数据预处理
去除重复值、去除“水军”评价、去除系统默认好评的评价

In [3]:
_=df['评价内容'].value_counts()[df['评价内容'].value_counts().values > 1].index
index=[]
for i,j in enumerate(df['评价内容'].values):
    if j in _:
        index.append(i)
df=df.drop(index)
df.drop_duplicates(subset='评价id',inplace=True)
index=np.arange(0,len(df))
df.index=index
# 保存数据
df.to_csv('./data/清洗后的数据.csv',index=False,encoding='utf_8_sig')

将每个评价的内容和其追评的内容拼接在一起，并保存起来

In [4]:
temp=[]
for i in range(len(df)):
    s=df['评价内容'].values[i].replace('\n','')
    if df['追评内容'].values[i] not in ['','此用户未填写评价内容']:
        s=s+df['追评内容'].values[i].replace('\n','')
    temp.append(s)
# 保存数据
pd.Series(temp).to_csv('./data/sentences.txt',index=False,header=False)

### 文本数据预处理

新词发现（信息熵模型）、并保存结果

In [5]:
f=open('./data/清洗后的数据.csv',encoding='utf-8')
df=pd.read_csv(f)
f.close()
df=df.fillna('')
comments=[]
for i in df['评价内容'].values:
    comments.append(i)
for i in df['追评内容'].values:
    if i == '此用户未填写评价内容':
        continue
    elif i == '':
        continue
    else:
        comments.append(i)
s=''.join(comments)
#定义要去掉的标点字
drop_dict = [u'，', u'\n', u'。', u'、', u'：', u'(', u')', u'[', u']', u'.', u',', u' ', u'\u3000', u'”', u'“', u'？', u'?', u'！', u'‘', u'’', u'…']
for i in drop_dict: #去掉标点字
    s = s.replace(i, '')
s=re.sub('&.*?;','',s)
s=re.sub('[^\u4e00-\u9fa50-9a-zA-Z]+','',s)
s=s.lower()

stop_file=open('./data/stoplist.txt','r',encoding='utf-8')
stoplist=[i.strip() for i in stop_file.readlines()]
stop_file.close()
min_count = 5 #录取词语最小出现次数
min_support = 20 #录取词语最低聚合度，1代表着随机组合
min_s = 2 #录取词语最低信息熵，越大说明越有可能独立成词
max_sep = 5 #候选词语的最大字数
t=[] #保存结果用。

temp=re.findall('[\u4e00-\u9fa5]|[0-9a-zA-Z]+',s)
t.append(pd.Series(temp).value_counts())
# tsum = t[0].sum() #统计总字数
rt = [] #保存结果用

for m in range(2, max_sep+1):
    print(u'正在生成%s字词...'%m)
    t.append([])
    hehe=[]
    if m == 2:
        for i in range(len(temp)-1):
            hehe.append(temp[i]+temp[i+1])
    elif m == 3:
        for i in range(len(temp)-2):
            hehe.append(temp[i]+temp[i+1]+temp[i+2])
    elif m == 4:
        for i in range(len(temp)-3):
            hehe.append(temp[i]+temp[i+1]+temp[i+2]+temp[i+3])
    elif m == 5:
        for i in range(len(temp)-4):
            hehe.append(temp[i]+temp[i+1]+temp[i+2]+temp[i+3]+temp[i+4])
    t[m-1] = t[m-1] + hehe
    t[m-1] = pd.Series(t[m-1]).value_counts() #逐词统计
    t[m-1] = t[m-1][t[m-1] > min_count] #最小次数筛选
    tt = t[m-1][:]
    for k in range(m-1):
        qq = np.array(list(map(lambda ms: np.log(t[m-1][ms]*t[m-2-k].sum()*t[k].sum()/t[m-2-k][''.join(re.findall('[\u4e00-\u9fa5]|[0-9a-zA-Z]+',ms)[:m-1-k])]/t[k][''.join(re.findall('[\u4e00-\u9fa5]|[0-9a-zA-Z]+',ms)[m-1-k:])]), tt.index))) > min_support #最小支持度筛选。
        tt = tt[qq]
    rt.append(tt.index)

def cal_S(sl): #信息熵计算函数
    return -((sl/sl.sum()).apply(np.log)*sl/sl.sum()).sum()

for i in range(2, max_sep+1):
    print(u'正在进行%s字词的最大熵筛选(%s)...'%(i, len(rt[i-2])))
    pp = [] #保存所有的左右邻结果
    if i == 2:
        for j in range(len(temp)-3):
            pp.append((temp[j],temp[j+1]+temp[j+2],temp[j+3]))
    elif i == 3:
        for j in range(len(temp)-4):
            pp.append((temp[j],temp[j+1]+temp[j+2]+temp[j+3],temp[j+4]))
    elif i == 4:
        for j in range(len(temp)-5):
            pp.append((temp[j],temp[j+1]+temp[j+2]+temp[j+3]+temp[j+4],temp[j+5]))
    elif i == 5:
        for j in range(len(temp)-6):
            pp.append((temp[j],temp[j+1]+temp[j+2]+temp[j+3]+temp[j+4]+temp[j+5],temp[j+6]))
    pp = pd.DataFrame(pp).set_index(1).sort_index() #先排序，这个很重要，可以加快检索速度
    index = np.sort(np.intersect1d(rt[i-2], pp.index)) #作交集
    #下面两句分别是左邻和右邻信息熵筛选
    index = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[0][s]).value_counts()), index))) > min_s]
    rt[i-2] = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[2][s]).value_counts()), index))) > min_s]

#下面都是输出前处理(排序+去停用词)
for i in range(len(rt)):
    temp=[]
    for j in rt[i]:
        if j in stoplist:
            temp.append(j)
    t[i+1] = t[i+1][rt[i]].sort_values(ascending = False)
    t[i+1].drop(temp,inplace=True)

#保存结果
pd.DataFrame(pd.concat(t[1:])).to_csv('./data/新词发现结果.txt', header = False, sep=' ')

正在生成2字词...
正在生成3字词...
正在生成4字词...
正在生成5字词...
正在进行2字词的最大熵筛选(1799)...
正在进行3字词的最大熵筛选(1311)...
正在进行4字词的最大熵筛选(909)...
正在进行5字词的最大熵筛选(260)...


将新词添加到现有的词典中（jieba自带的词典）

In [6]:
# 现有词典
old_file=open('./data/dict.txt','r',encoding='utf-8')
old_dict=[i.split(' ')[0] for i in old_file.readlines()]
old_file.close()

# 新词发现结果
new_file=open('./data/新词发现结果.txt','r',encoding='utf-8')
new_dict={i.split(' ')[0]:i.split(' ')[1].strip() for i in new_file.readlines()}
new_file.close()

# 合并
with open('./data/dict.txt','a+',encoding='utf-8') as f:
    for word,fred in new_dict.items():
        if word not in old_dict:  # 新词才添加进去
            temp=word+' '+fred
            f.write(temp)
            f.write('\n')

分词、并去除停用词，并保存结果

In [7]:
def process(s):
    _=set()  # 表情集合
    for i in emoji.emoji_lis(s):
        _.add(emoji.UNICODE_EMOJI.get(i['emoji']))
    emo=[emoji.emojize(i) for i in list(_)]
    w=re.sub('&[a-zA-Z]+;|<br/>|电池续航：|拍照效果：|显示效果：|运行速度：|外形外观：|屏幕音效：|其他特色：|通信音质：|待机时间：','',s)
    w=re.sub('[^\u4e00-\u9fa50-9a-zA-Z]+','',w)  # 删除所有非中文、英文、数字
    w=w.lower()
    temp=jieba.lcut(w) # 分词
    result=[i for i in temp if i not in stoplist] # 去除停用词
    result=result+emo
    return ' '.join(result)

stop_file=open('./data/stoplist.txt','r',encoding='utf-8') # 停用词表
stoplist=[i.strip() for i in stop_file.readlines()]
stop_file.close()
jieba.load_userdict('./data/dict.txt')  # 加载自定义词典
# 增加词语
ws=['天空之境','亮黑色','极光色','珠光贝母','赤茶橘','墨玉蓝','嫣紫色','差评','中评','京东商城','苏宁易购','华为商城']
for i in ws:
    jieba.add_word(i)
all_comments=pd.read_csv('./data/sentences.txt',header=None,names=['原评论'])
all_comments['分词处理后的评论']=all_comments['原评论'].apply(lambda s:process(s))
all_comments.to_csv('./data/分词后的评论数据.csv',index=False,encoding='utf_8_sig')

Building prefix dict from the default dictionary ...
Dumping model to file cache D:/python3.7/jieba.cache
Loading model cost 1.444 seconds.
Prefix dict has been built succesfully.


In [8]:
all_comments

,原评论,分词处理后的评论
0,买个妈妈的，她很满意,买个 妈妈 满意
1,已经买入第二台了 质量真不错,买入 第二台 质量 真不错
2,发货很快，拍照确实很清楚，通话很清晰，屏幕显示效果很好，手机运行非常不错，已第2次在他家买手...,发货 很快 拍照 确实 通话 清晰 屏幕显示 效果 手机 运行 不错 次 他家 买手机
3,音质：第二次买了，帮朋友买的，价格降了，产品一样的好，好评！全5分。,音质 第二次 帮 朋友 价格 降 产品 好评 全 分
4,还不错，挺漂亮的，拍照效果杠杠滴，送老公的，快递很快哦，包装很好,不错 漂亮 拍照 效果 杠杠滴 送 老公 快递 很快 包装
5,送给自己的女神节礼物哦(&acute;-&omega;-`！很喜欢！用了几天感觉不错！人脸识...,送给 女神 节 礼物 喜欢 几天 感觉 不错 人脸识别 24 期 免息 轻松 拥有 hiah...
6,买贵了，现在总体体验好,买贵 总体 体验
7,用了一段时间。充电最快。反应还可以。屏幕有点花纹，只有太阳底下看的到，平时不仔细看看不出来。...,一段时间 充电 最快 屏幕 花纹 太阳 底下 平时 仔细 找到 门店 人员 看不出来 只能 做 摆
8,快递小哥送货很快,快递 小哥 送货 很快
9,其他特色：超级好(✪▽✪ 很喜欢 运行很快 非常棒 物流一天就到了 好快啊啊啊啊啊啊！,超级 喜欢 运行 很快 棒 物流 啊啊啊 啊啊啊
